Inga Ulusoy, Computational modelling in python, SoSe2020 

# The quantum harmonic oscillator II

Reference:
https://doi.org/10.1021/acs.jchemed.7b00003

The quantum harmonic oscillator is another standard problem in Quantum Mechanics. However, contrary to the particle in a box, it is widely and very successfully used as a model to describe molecular vibrations. For example, all standard quantum chemistry programs use the harmonic oscillator approximation in their frequency analysis. The advantage of this approximation is that the knowledge of the second derivatives at the optimized structure of a molecule is sufficient to calculate the IR frequencies with which that molecule vibrates. These frequencies can then be compared to spectral data and help identify the type of bonds and structural motif (isomer) of a molecule.

The potential energy term is approximated as a harmonic potential, where in the classical analogue the force acting on the particle is directly proportional to the displacement from equilibrium
\begin{align}
V = \frac{1}{2} k x^2
\end{align}
with the force constant $k=\mu \omega^2$ for a reduced mass $\mu$ and vibrational frequency $\omega$. The complete Hamiltonian thus reads
\begin{align}
\hat{H} = \hat{T} + V(x) = -\frac{\hbar^2}{2\mu} \frac{\partial^2}{\partial x^2} + \frac{1}{2} k x^2
\end{align}
As an example, we will look at HCl with the reduced mass 
\begin{align}
\mu =\frac{m_H m_{Cl}}{m_H+m_{Cl}}
\end{align}
First, we need to define and calculate some basic parameters and functions. We will use the physical constants from scipy; for a comprehensive list see https://docs.scipy.org/doc/scipy/reference/constants.html.

It is more convenient and numerically accurate to use atomic units. Furthermore, we will use mass-weighted coordinates $R=x/\sqrt{\mu}$, leading to the more compact Hamiltonian
\begin{align}
\hat{H} = \hat{T} + V(R) = -\frac{\hbar^2}{2} \frac{\partial^2}{\partial R^2} + \frac{1}{2} \omega^2 R^2
\end{align}

Now, we will look at the time-dependent Schrödinger equation (TDSE).

In [ ]:
from numpy import *
from scipy import linalg
from scipy.constants import eV, c, h, hbar, m_e
from scipy.constants import physical_constants
import matplotlib.pyplot as plt
prop_cycle = plt.rcParams['axes.prop_cycle']
colors = prop_cycle.by_key()['color']
amu = physical_constants['atomic mass constant'][0]
cm2Eh = 4.556335E-6
au2fs = 0.0241888

In [ ]:
#parameters of the HO: no of grid points for x
nsteps=500
xmin = 0
xmax = 5
#vibrational frequency in cm^-1
w = 2990
#interatomic distance in a.u.
R0 = 2.9
omega = w*cm2Eh

def get_mu(m1,m2):
    mu = m1*m2/(m1+m2)
    return mu

def get_k(mu,omega):
    k = mu*(2*pi*omega*100*c)**2
    #print(omega*100*c*10**-12,'THz')
    return k

m1 = 1*amu
m2 = 35*amu
mu = get_mu(m1,m2)
#convert this to atomic units by dividing througe m_e
muau=mu/m_e
k = get_k(mu,w)
print('Reduced mass is {:3.2e} kg ({:4.2f} au) and the force constant {:4.2f} N/m.'.format(mu,muau,k))

#set up the nuclear grid
#step size h for the numerical derivative
xgrid,h=linspace(xmin,xmax,nsteps,retstep=True)

def potential(x,omega):
    pot=(0.5*omega**2*x**2)
    return pot

# create the Hamiltonian
Hamiltonian=zeros((nsteps,nsteps))
[i,j] = indices(Hamiltonian.shape)
Laplacian=(-2.0*diag(ones(nsteps))+diag(ones(nsteps-1),1)+diag(ones(nsteps-1),-1))/(float)(h**2)
#convert the internuclear distance to relative distance from R0
xgridr = (xgrid-R0)*sqrt(muau)
V=potential(xgridr,omega)
Hamiltonian[i==j]=V
Hamiltonian+=(-1.0/(2.0*muau))*Laplacian

energies, wavef = linalg.eigh(Hamiltonian)
print('Zero-point energy: {:3.2f} cm-1'.format(energies[0]/cm2Eh))
print('Fundamental vibration: {:3.2f} cm-1'.format((energies[1]-energies[0])/cm2Eh))

Let's look at the wave functions.

In [ ]:
#here goes a plot
plt.plot(wavef[:,0])
plt.plot(wavef[:,1])
plt.plot(wavef[:,2])

plt.show()

In [ ]:
#get the expectation values
class expect:
    '''This is the expectation value class.'''
    def __init__(self, wave, xgrid):
        self.wave = wave
        self.states = len(wave[0])
        print(self.states)
        self.xgrid = xgrid
    
    def x(self):
        xval = zeros(self.states)
        for i in range(self.states):
            xval[i] = sum(conj(self.wave[:,i])*self.xgrid*self.wave[:,i])
        return xval

    def p(self):
        pval = zeros((self.states),dtype=complex)
        h=self.xgrid[1]-self.xgrid[0]
        for i in range(self.states):
            pgrid= 1j * gradient(self.wave[:,i],h)
        #plt.plot(self.xgrid,real(pgrid))
        #plt.plot(self.xgrid,imag(pgrid))
        #plt.plot(self.xgrid,self.wave)
            pval[i] = sum(conj(self.wave[:,i])*pgrid)
        return pval

# Time propagation

Please note:
https://stackoverflow.com/questions/34577870/using-scipy-integrate-complex-ode-instead-of-scipy-integrate-ode

"Complex ode is broken - this would appear to be a known bug in scipy.integrate. It seems that additional argument passing is broken in complex_ode. You could try and see if they have fixed it in a newer release (although this bug report suggests they haven't), or just restrict yourself to your own wrapper functions without additional arguments when using complex_ode." 

That is why we are using zvode.

About zvode: "Warning - This integrator is not re-entrant. You cannot have two ode instances using the “zvode” integrator at the same time."


In [ ]:
from scipy.integrate import ode
#ode and odeint cannot handle matrices (only vectors).
#so we need to flatten the matrices to vectors for the implementation.
def f(t, y, arg1):
    return [-1j*arg1*y] 

#set the inital state
initstate = 1
val = 1.0
psi=wavef.flatten(order='F')
energy=zeros(nsteps*nsteps)
cvec = zeros(nsteps*nsteps)
cvec[(initstate-1)*nsteps:initstate*nsteps]=val
for i in range(nsteps):
    for j in range(nsteps):
        energy[i*nsteps+j] = energies[i]

psi=psi*cvec
t0=0.0
t1fs = 10
dtfs = 1
tsteps = int(t1fs/dtfs)
t1 = t1fs/au2fs
dt = dtfs/au2fs

r = ode(f).set_integrator('zvode', method='adams')
r.set_initial_value(psi, t0).set_f_params(energy)
    
psitemp=zeros((tsteps,nsteps*nsteps),dtype=complex)
tgrid=zeros(tsteps)
i=0
while r.successful() and r.t < t1:
    #print(r.t+dt, r.integrate(r.t+dt))
    tgrid[i]=r.t+dt
    psitemp[i]=r.integrate(r.t+dt)
    i+=1

psit = psitemp.reshape(tsteps,nsteps,nsteps)
#build one psi
psiout = zeros((tsteps,nsteps),dtype=complex)
# Psi = Psi0, Psi1, Psi2 ,.. up to all eigenstates
for i in range(tsteps):
    for j in range(nsteps):
        psiout[i] += psit[i,j]
        #psiout = Psi0 + Psi1 + Psi2,.. 

In [ ]:
#plot and compare to analytical solution
fig, ax = plt.subplots(tsteps,2,figsize=(10,3*tsteps),sharex=True,sharey=True)
for i in range(tsteps):
    ax[i,0].plot(xgrid,real(psiout[i]),label='real part')
    ax[i,0].plot(xgrid,imag(psiout[i]),label='imaginary part')
    ax[i,0].text(xmin,0.17,'t = {:2.1f}fs'.format(tgrid[i]*au2fs),fontsize=14)
    ax[i,0].set_ylim(bottom=-0.2,top=0.2)
    ax[i,1].plot(xgrid,real(wavef[:,initstate-1]*exp(-1j * energies[initstate-1] * tgrid[i])),label='real part')
    ax[i,1].plot(xgrid,imag(wavef[:,initstate-1]*exp(-1j * energies[initstate-1] * tgrid[i])),label='imaginary part')
    
ax[0,0].set_title(r'$\Psi_{ODE}$',fontsize=18)
ax[0,1].set_title(r'$\Psi_{ex}$',fontsize=18)

plt.subplots_adjust(wspace = 0.0)
plt.subplots_adjust(hspace = 0.0)

plt.show()

In [ ]:
#let's try a superposition state
#set the inital state
initstate = [1,2]
val = 1.0/sqrt(len(initstate))
#or set this manually if you do not want a 1:1 superposition
psi=wavef.flatten(order='F')
energy=zeros(nsteps*nsteps)
cvec = zeros((nsteps*nsteps),dtype=complex)
for i in range(nsteps):
    for j in range(nsteps):
        energy[i*nsteps+j] = energies[i]

for i in range(len(initstate)):
    cvec[(initstate[i]-1)*nsteps:initstate[i]*nsteps]=val
psi=psi*cvec
t0=0.0
t1fs = 10
dtfs = 1
tsteps = int(t1fs/dtfs)
t1 = t1fs/au2fs
dt = dtfs/au2fs
def f(t, y, arg1):
    return [-1j*arg1*y] 
r = ode(f).set_integrator('zvode', method='adams')
r.set_initial_value(psi, t0).set_f_params(energy)
    
psitemp=zeros((tsteps,nsteps*nsteps),dtype=complex)
tgrid=zeros(tsteps)
i=0
while r.successful() and r.t < t1:
    tgrid[i]=r.t+dt
    psitemp[i]=r.integrate(r.t+dt)
    i+=1

psit = psitemp.reshape(tsteps,nsteps,nsteps)
#build one psi
psiout = zeros((tsteps,nsteps),dtype=complex)
for i in range(tsteps):
    for j in range(nsteps):
        psiout[i] += psit[i,j]

In [ ]:
#plot and compare to analytical solution
fig, ax = plt.subplots(tsteps,2,figsize=(10,3*tsteps),sharex=True,sharey=True)
for i in range(tsteps):
    ax[i,0].plot(xgrid,real(psiout[i]),label='real part')
    ax[i,0].plot(xgrid,imag(psiout[i]),label='imaginary part')
    ax[i,0].plot(xgrid,abs(psiout[i])**2,label='density')
    ax[i,0].text(xmin,0.17,'t = {:2.1f}fs'.format(tgrid[i]*au2fs),fontsize=14)
    ax[i,0].set_ylim(bottom=-0.2,top=0.2)
    ax[i,1].plot(xgrid,real(val*wavef[:,initstate[0]-1]*exp(-1j * energies[initstate[0]-1] * tgrid[i]) \
                           +val*wavef[:,initstate[1]-1]*exp(-1j * energies[initstate[1]-1] * tgrid[i])),label='real part')
    ax[i,1].plot(xgrid,imag(val*wavef[:,initstate[0]-1]*exp(-1j * energies[initstate[0]-1] * tgrid[i]) \
                           +val*wavef[:,initstate[1]-1]*exp(-1j * energies[initstate[1]-1] * tgrid[i])),label='imaginary part')
ax[0,0].set_title(r'$\Psi_{ODE}$',fontsize=18)
ax[0,1].set_title(r'$\Psi_{ex}$',fontsize=18)

plt.subplots_adjust(wspace = 0.0)
plt.subplots_adjust(hspace = 0.0)
plt.show()

In [ ]:
#let's try a superposition state
#set the inital state
initstate = [1,2,3]
val = 1.0/sqrt(len(initstate))
#or set this manually if you do not want a 1:1 superposition
psi=wavef.flatten(order='F')
energy=zeros(nsteps*nsteps)
cvec = zeros((nsteps*nsteps),dtype=complex)
for i in range(nsteps):
    for j in range(nsteps):
        energy[i*nsteps+j] = energies[i]

for i in range(len(initstate)):
    cvec[(initstate[i]-1)*nsteps:initstate[i]*nsteps]=val
psi=psi*cvec
t0=0.0
t1fs = 10
dtfs = 1
tsteps = int(t1fs/dtfs)
t1 = t1fs/au2fs
dt = dtfs/au2fs
def f(t, y, arg1):
    return [-1j*arg1*y] 
r = ode(f).set_integrator('zvode', method='adams')
r.set_initial_value(psi, t0).set_f_params(energy)
    
psitemp=zeros((tsteps,nsteps*nsteps),dtype=complex)
tgrid=zeros(tsteps)
i=0
while r.successful() and r.t < t1:
    tgrid[i]=r.t+dt
    psitemp[i]=r.integrate(r.t+dt)
    i+=1

psit = psitemp.reshape(tsteps,nsteps,nsteps)
#build one psi
psiout = zeros((tsteps,nsteps),dtype=complex)
for i in range(tsteps):
    for j in range(nsteps):
        psiout[i] += psit[i,j]

In [ ]:
#plot and compare to analytical solution
fig, ax = plt.subplots(tsteps,figsize=(5,3*tsteps),sharex=True,sharey=True)
for i in range(tsteps):
    ax[i].plot(xgrid,real(psiout[i]),label='real part')
    ax[i].plot(xgrid,imag(psiout[i]),label='imaginary part')
    ax[i].plot(xgrid,abs(psiout[i])**2,label='density')
    ax[i].text(xmin,0.17,'t = {:2.1f}fs'.format(tgrid[i]*au2fs),fontsize=14)
    ax[i].set_ylim(bottom=-0.2,top=0.2)
ax[0].set_title(r'$\Psi_{ODE}$',fontsize=18)

plt.subplots_adjust(wspace = 0.0)
plt.subplots_adjust(hspace = 0.0)
plt.show()

# Optional task
Extent your program to TDSE propagation using the `zvode` solver with eigenstates as solutions from the TISE as above (and not analytical solutions). The routine should be able to use the particle in a box and the quantum HO (AHO) eigenfunctions.